In [2]:
import os
import pandas as pd
import logging.config
import matplotlib.pyplot as plt
import numpy as np
from cycler import cycler #for matplotlib colors
import seaborn as sns
from sklearn import preprocessing
from sqlalchemy import create_engine



LOGGING_CONFIG = os.getenv('LOGGING_CONFIG')
logging.config.fileConfig(LOGGING_CONFIG)
logger = logging.getLogger('mainstream_data')

DATADIR = os.getenv('DATADIR')
ENGINE = os.getenv('ENGINE')
logger.info("using data from {}".format(ENGINE))

2018-05-09 18:22:41,408 - mainstream_data - INFO - using data from postgresql://ellieking@localhost:5432/givedata_180504


### Read in data

In [ ]:
# ### Read in data
engine = create_engine(ENGINE)

logger.info('Importing facts_metrics')
facts_metrics = pd.read_sql_query('select * from "facts_metrics"',con=engine)
logger.info('Importing dates')
dates = pd.read_sql_query('select * from "dimensions_dates"',con=engine)
logger.info('Dropping {} dates duplicates'.format(sum(dates.duplicated())))
dates = dates.drop_duplicates()
logger.info('Importing items')
items = pd.read_sql_query('select * from "dimensions_items"',con=engine)

2018-05-09 18:22:41,884 - mainstream_data - INFO - Importing facts_metrics


In [ ]:
items.columns

In [ ]:
mainstream_doctypes = ['answer',
                       'completed_transaction', 
                       'guide', 
                       'help_page', 
                       'licence', 
                       'local_transaction', 
                       'place', 
                       'simple_smart_answer', 
                       'transaction']
mainstream = items.loc[items['document_type'].isin(mainstream_doctypes)]

### Join facts_metrics to specific item variables

In [ ]:
mainstream_facts_metrics = pd.merge(
    left=facts_metrics,
    right=mainstream,
    how='inner',
    left_on='dimensions_item_id', # 
    right_on='id'
)

In [ ]:
mainstream_facts_metrics = mainstream_facts_metrics.drop(['id_x', 'id_y', 'created_at_x',
       'updated_at_x', 'created_at_y',
       'updated_at_y'], axis=1).copy()

In [ ]:
mainstream_facts_metrics.shape

In [ ]:
mainstream_facts_metrics.to_csv('DATA/mainstream_content.csv.gz', compression='gzip')

# Get row per content ID and data from 11APR18

In [ ]:
wed11 = facts_metrics[facts_metrics['dimensions_date_id']==(datetime.date(2018, 4, 14))]

In [ ]:
wed11.shape

In [ ]:
wed11 = wed11.drop('id', axis=1)

In [ ]:
wed11.head()

In [ ]:
facts_metrics.shape

In [ ]:
content_wed11 = pd.merge(wed11, mainstream, how='inner', 
                         left_on='dimensions_item_id', 
                         right_on='id')

In [ ]:
content_wed11.shape

In [ ]:
content_wed11.tail(20)

In [ ]:
content_wed11.to_csv('DATA/mainstream_content_weds_11APR18.csv.gz', compression='gzip')

In [ ]:
content_wed11.shape

In [ ]:
content_wed11.content_id.nunique()